# ライブラリ一覧

In [ ]:
!pip install bs4
!pip install pandas
!pip install pykakasi


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for jaconv: filename=jaconv-0.4.0-py3-none-any.whl size=18313 sha256=78654c646619838e25525addf43394417b9d26b803d509bbcaaa88f16eb89886
  Stored in directory: /Users/ryota/Library/Caches/pip/wheels/c9/97/53/1f827ebf916b899520ce663227497206525c432be72cc29265
Successfully built jaconv
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pykakasi]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


# Tools

In [2]:
import pykakasi
import re

# 初期化（ふりがな→ローマ字変換器）
kks = pykakasi.kakasi()
kks.setMode("H", "a")  # ひらがな to ローマ字
kks.setMode("K", "a")  # カタカナ to ローマ字
kks.setMode("J", "a")  # 漢字 to ローマ字
kks.setMode("r", "Hepburn")  # ヘボン式
converter = kks.getConverter()

# 正規化関数
def normalize(text):
    text = text.replace("\u3000", " ")       # 全角スペース → 半角
    text = re.sub(r"\s+", " ", text)         # 複数空白 → 半角スペース1つ
    return text.strip()

# ローマ字変換関数
def to_romaji(text):
    return converter.do(text).capitalize()

/var/folders/k3/6dyshyvn29d6tl28wnz4r2m40000gn/T/ipykernel_22308/2820461022.py:6: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kks.setMode("H", "a")  # ひらがな to ローマ字
/var/folders/k3/6dyshyvn29d6tl28wnz4r2m40000gn/T/ipykernel_22308/2820461022.py:7: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kks.setMode("K", "a")  # カタカナ to ローマ字
/var/folders/k3/6dyshyvn29d6tl28wnz4r2m40000gn/T/ipykernel_22308/2820461022.py:8: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kks.setMode("J", "a")  # 漢字 to ローマ字
/var/folders/k3/6dyshyvn29d6tl28wnz4r2m40000gn/T/ipykernel_22308/2820461022.py:9: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kks.setMode("r", "Hepburn")  # ヘボン式
/var/folders/k3/6dyshyvn

# 衆議院

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

OUTDIR = "../data/syu_person"  # 出力ディレクトリ
if not os.path.exists(OUTDIR):
    os.makedirs(OUTDIR)  # 出力ディレクトリが存在しない場合は作成する

# csvをマージするための変数
merged_data = []

# 国会議員名簿のHTMLをスクレイピングしてCSVに変換するスクリプト
# あ行からわ行までの議員名簿を順に取得し、CSVファイルに保存する
for i in range(1, 11):
    URL = f"https://www.shugiin.go.jp/internet/itdb_annai.nsf/html/statics/syu/{i}giin.htm"
    OUTPUT_FILE = URL.split("/")[-1].replace(".htm", ".csv")  # URLからファイル名を取得

    response = requests.get(URL)
    response.encoding = response.apparent_encoding  # 文字化け対策

    soup = BeautifulSoup(response.text, "html.parser")

    data = []
    rows = soup.find_all("tr", valign="top")

    for row in rows:
        cols = row.find_all("td")
        if len(cols) == 5:
            name = normalize(cols[0].get_text(strip=True).replace("君", ""))
            kana = normalize(cols[1].get_text(strip=True))
            kana_parts = kana.split(" ", 1)
            kana_last = kana_parts[0]
            kana_first = kana_parts[1] if len(kana_parts) > 1 else ""

            romaji_last = to_romaji(kana_last)
            romaji_first = to_romaji(kana_first)

            data.append({
                "氏名": name,
                "ふりがな（姓）": kana_last,
                "ふりがな（名）": kana_first,
                "ローマ字（姓）": romaji_last,
                "ローマ字（名）": romaji_first,
                "会派": normalize(cols[2].get_text(strip=True)),
                "選挙区": normalize(cols[3].get_text(strip=True)),
                "当選回数": normalize(cols[4].get_text(strip=True)),
            })

    # DataFrame に変換してCSV保存
    df = pd.DataFrame(data)
    df.to_csv(OUTDIR + "/" + OUTPUT_FILE, index=False, encoding="utf-8-sig")

    print("CSVファイルを出力しました：", OUTDIR + "/" + OUTPUT_FILE)

    # csvファイルをマージする
    merged_data.append(df)

# マージしたデータを1つのDataFrameにまとめる
merged_df = pd.concat(merged_data, ignore_index=True)
merged_df.to_csv(OUTDIR + "/merged_syu_person.csv", index=False, encoding="utf-8-sig")


/var/folders/k3/6dyshyvn29d6tl28wnz4r2m40000gn/T/ipykernel_22308/2820461022.py:20: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  return converter.do(text).capitalize()


CSVファイルを出力しました： ../data/syu_person/1giin.csv
CSVファイルを出力しました： ../data/syu_person/2giin.csv
CSVファイルを出力しました： ../data/syu_person/3giin.csv
CSVファイルを出力しました： ../data/syu_person/4giin.csv
CSVファイルを出力しました： ../data/syu_person/5giin.csv
CSVファイルを出力しました： ../data/syu_person/6giin.csv
CSVファイルを出力しました： ../data/syu_person/7giin.csv
CSVファイルを出力しました： ../data/syu_person/8giin.csv
CSVファイルを出力しました： ../data/syu_person/9giin.csv
CSVファイルを出力しました： ../data/syu_person/10giin.csv


# 参議院

In [4]:
URL = "https://www.sangiin.go.jp/japanese/joho1/kousei/giin/218/giin.htm"


OUTDIR = "../data/san_person"  # 出力ディレクトリ
if not os.path.exists(OUTDIR):
    os.makedirs(OUTDIR)  # 出力ディレクトリが存在しない場合は作成する
OUTPUT_FILE = URL.split("/")[-1].replace(".htm", ".csv")  # URLからファイル名を取得

response = requests.get(URL)
response.encoding = response.apparent_encoding  # 文字化け対策
soup = BeautifulSoup(response.text, "html.parser")


# 対象表の特定
tables = soup.find_all("table")
target = None
for table in tables:
    if any("議員氏名" in th.get_text() for th in table.find_all("th")):
        target = table
        break
if not target:
    raise RuntimeError("対象の表が見つかりません")

data = []
current = None
for tr in target.find_all("tr"):
    th = tr.find("th")
    tds = tr.find_all("td")
    if th and not tds:
        current = normalize(th.get_text())
        continue
    if len(tds) >= 5:
        name = normalize(tds[0].get_text())
        reading = normalize(tds[1].get_text())
        party = normalize(tds[2].get_text())
        district = normalize(tds[3].get_text())
        term = normalize(tds[4].get_text())
        # 読み方を姓・名に分割
        parts = reading.split(" ")
        last_h = parts[0] if parts else ""
        first_h = parts[1] if len(parts) > 1 else ""
        # ローマ字変換
        last_r = converter.do(last_h) if last_h else ""
        first_r = converter.do(first_h) if first_h else ""
        row = {
            "行区分": current,
            "氏名": name,
            "姓ふりがな": last_h,
            "名ふりがな": first_h,
            "姓ローマ字": last_r,
            "名ローマ字": first_r,
            "会派": party,
            "選挙区": district,
            "任期満了": term
        }
        data.append(row)

df = pd.DataFrame(data)
print(df.head())

# 必要に応じてCSVに保存
df.to_csv(OUTDIR + "/" + OUTPUT_FILE, index=False, encoding="utf-8-sig")

  行区分     氏名 姓ふりがな 名ふりがな     姓ローマ字      名ローマ字  会派    選挙区        任期満了
0       青木 愛   あおき    あい      aoki         ai  立憲     比例  令和10年7月25日
1      青木 一彦   あおき  かずひこ      aoki   kazuhiko  自民  鳥取・島根  令和10年7月25日
2      青島 健太  あおしま   けんた   aoshima      kenta  維新     比例  令和10年7月25日
3      青山 繁晴  あおやま  しげはる    aoyama  shigeharu  自民     比例  令和10年7月25日
4       赤松 健  あかまつ    けん  akamatsu        ken  自民     比例  令和10年7月25日


/var/folders/k3/6dyshyvn29d6tl28wnz4r2m40000gn/T/ipykernel_22308/1383196681.py:43: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  last_r = converter.do(last_h) if last_h else ""
/var/folders/k3/6dyshyvn29d6tl28wnz4r2m40000gn/T/ipykernel_22308/1383196681.py:44: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  first_r = converter.do(first_h) if first_h else ""
